In [1]:
import pandas as pd
import os
import urllib

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By

In [2]:
location = str(os.getcwd())
location = location[:-23]
article_df = pd.read_csv(location + "Articles/articles_cleaned.csv")

#just testing iub right now
article_df = article_df.to_dict('records')

pd.DataFrame(article_df)

,Unnamed: 0,url,date,body,title,location
0,0,https://www.idsnews.com/article/2011/12/office...,12/11/2011,"For almost 40 years, the Office for Women’s Af...","Office for Women’s Affairs works to educate, e...",IUB
1,1,https://www.idsnews.com/article/2011/12/iusa-r...,12/11/2011,With two-thirds of its time in office already ...,IUSA remembers semester,IUB
2,2,https://www.idsnews.com/article/2011/12/local-...,12/1/2011,Local students at Bloomington High School Nort...,Local musicians to play as part of benefit con...,IUB
3,3,https://www.idsnews.com/article/2011/11/not-th...,11/7/2011,I’ve lived in Bloomington my entire life. I wa...,Not the Bloomington I know,IUB
4,4,https://www.idsnews.com/article/2011/11/letter...,NaN,Using rankings is a great marketing tool used ...,Letters to the editor,IUB
...,...,...,...,...,...,...
11116,11116,https://www.dailycardinal.com/article/2021/10/...,10/1/2021,Enthusiastic clapping echoed throughout the he...,ASM passes legislation intended to restore sha...,Wisconsin
11117,11117,https://www.dailycardinal.com/article/2021/09/...,9/16/2021,"For some of us in the world, the fight to belo...",Privilege and Power,Wisconsin
11118,11118,https://www.dailycardinal.com/article/2021/04/...,4/27/2021,"Every great musician is one of a kind, but the...",'The United States Vs. Billie Holiday' proves ...,Wisconsin
11119,11119,https://www.dailycardinal.com/article/2021/04/...,4/22/2021,What is the point of taking history class? Thi...,Boycott the 2022 Beijing Olympics,Wisconsin


In [3]:
#Selenium Gecko Settings
options = webdriver.FirefoxOptions()
options.binary = 'C:/Program Files/Mozilla Firefox/firefox.exe'
path_to_wkhtmltopdf = r'C:/Program Files/wkhtmltopdf/bin/wkhtmltopdf.exe'

def collect_imgage(url, img_out_path, location):
    driver = webdriver.Firefox(options=options,  service=Service(GeckoDriverManager().install()))
    driver.get(url)
    
    ##IU
    if location == "IUB" or location == "Wisconsin":
        images = driver.find_elements(By.CLASS_NAME, "dom-art-container")
    
    ##Stanford
    elif location == "Stanford":
        images = driver.find_elements(By.CLASS_NAME, "wp-caption")
        
        
    ##UMD
    elif location == "UMD":
        images = driver.find_elements(By.CLASS_NAME, "mt-3")
    
    ##UMN
    elif location == "UMN":
        images = driver.find_elements(By.CLASS_NAME, "modal-photo")
    
    images_found = len(images)
    
    if images_found > 0:
        if len(images[0].find_elements(By.TAG_NAME, "img")) > 0:
            image = images[0].find_element(By.TAG_NAME, "img").screenshot_as_png
            with open(img_out_path, 'wb') as file:
                file.write(image)
            global successes
            successes += 1
    else:
        global failures
        failures += 1
        
        
    driver.close()
    return(images_found)

In [4]:
image_info = []

i = 0
for article in tqdm(article_df):
    #format date for image_id
    date = ""        
    for item in (str(article["date"]).split("/")):
        if len(item) < 2:
            item = "0" + item
        date = date + item
    
    img_out_path = location + "Analyses/Visual Framing/images/" + article["location"] + "_" + date + "_" + str(i) + ".png"
    url = article["url"]
    
    #download image
    num_images = collect_imgage(url, img_out_path, article["location"])
    
    
    #unpack info
    if num_images > 0:
        row = {}
        row["date"] = article["date"]
        row["location"] = article["location"]
        row["url"] = article["url"]
        
        
        image_id = row["location"] + "_" + date + "_" + str(i)
        
        row["image_file"] = image_id + ".png"

    
        image_info.append(row)
        
    i += 1

 17%|████████████▌                                                           | 1943/11121 [6:15:31<29:33:52, 11.60s/it]


KeyboardInterrupt: 

In [5]:
pd.DataFrame(image_info).to_csv("image_info.csv", index=False)

In [ ]:
print(successes)
print(failures)